# Übernahme der Daten aus OSM Protobuf, da hier alle Tages enthalten sind

In [2]:
import duckdb
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

## Url zurGeofabrik und lokale Ablage (zur Beschleunigung)

In [3]:
url = 'https://download.geofabrik.de/europe/germany/niedersachsen-latest.osm.pbf'
output_path = "downloads/niedersachsen-latest.osm.pbf"

In [ ]:
# Download a PDF file to the downloads folder
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)
print(f"Downloaded file to {output_path}")

Downloaded file to downloads/niedersachsen-latest.osm.pbf


In [4]:
duck = duckdb.connect(database=':memory:')

In [6]:
duck.sql("""INSTALL spatial;
            LOAD spatial;"""
         )

In [7]:
duck.sql(f"DESCRIBE SELECT * FROM ST_READOSM('{output_path}');")

┌─────────────┬──────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                 column_type                  │  null   │   key   │ default │  extra  │
│   varchar   │                   varchar                    │ varchar │ varchar │ varchar │ varchar │
├─────────────┼──────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ kind        │ ENUM('node', 'way', 'relation', 'changeset') │ YES     │ NULL    │ NULL    │ NULL    │
│ id          │ BIGINT                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ tags        │ MAP(VARCHAR, VARCHAR)                        │ YES     │ NULL    │ NULL    │ NULL    │
│ refs        │ BIGINT[]                                     │ YES     │ NULL    │ NULL    │ NULL    │
│ lat         │ DOUBLE                                       │ YES     │ NULL    │ NULL    │ NULL    │
│ lon         │ DOUBLE                                       │ YES     │ 

In [31]:
duck.sql(f"""create or replace table matching_ways as 
         select 
         id, tags['name'] as name, 
         tags['highway'] as highway, tags['maxspeed'] as maxspeed, tags['oneway'] as oneway, tags['oneway:bus'] as oneway_bus, tags['area'] as area,  tags['access'] as access, refs
        
         from ST_READOSM('{output_path}')
          where tags['highway'] is not null and kind = 'way'
          -- LIMIT 500;         
         """)

In [32]:
duck.sql("""from matching_ways""")

┌─────────┬───────────────────────┬──────────────┬──────────┬─────────┬────────────┬─────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│   id    │         name          │   highway    │ maxspeed │ oneway  │ oneway_bus │  area   │ access  │                                                                                                refs                                                                                                │
│  int64  │        varchar        │   varchar    │ varchar  │ varchar │  varchar   │ varchar │ varchar │                                                                                              int64[]                                                                                               │
├─────────┼───────────────────────┼──────────────┼──────────┼─────────┼────────────┼─────────┼

In [33]:
duck.sql(f"""
         CREATE OR REPLACE TEMP TABLE matching_ways_with_nodes_refs AS
SELECT id, UNNEST(refs) as ref, UNNEST(range(length(refs))) as ref_idx
FROM ST_READOSM('{output_path}')
SEMI JOIN matching_ways USING (id)
WHERE kind = 'way';

SELECT * FROM matching_ways_with_nodes_refs order by id, ref_idx;
         """)

┌─────────┬─────────────┬─────────┐
│   id    │     ref     │ ref_idx │
│  int64  │    int64    │  int64  │
├─────────┼─────────────┼─────────┤
│ 2387253 │    10373627 │       0 │
│ 2387253 │ 11750239882 │       1 │
│ 2387253 │    10373682 │       2 │
│ 2387253 │    10373626 │       3 │
│ 2387253 │    10373681 │       4 │
│ 2387253 │  1188962150 │       5 │
│ 2387253 │    10373633 │       6 │
│ 2387253 │  1192335300 │       7 │
│ 2387253 │    10376971 │       8 │
│ 2387253 │    10373625 │       9 │
│    ·    │        ·    │       · │
│    ·    │        ·    │       · │
│    ·    │        ·    │       · │
│ 4277620 │    25751731 │       5 │
│ 4277620 │   301662223 │       6 │
│ 4277620 │   301662206 │       7 │
│ 4277620 │    25751734 │       8 │
│ 4277620 │  6418549371 │       9 │
│ 4277620 │  6418549373 │      10 │
│ 4277620 │    25751735 │      11 │
│ 4277620 │  6418549369 │      12 │
│ 4277620 │  6418549367 │      13 │
│ 4277620 │  6178423185 │      14 │
├─────────┴─────────────┴───

In [34]:
duck.sql(f"""CREATE or replace TEMP TABLE required_nodes_with_geometries AS
SELECT id, ST_POINT(lon, lat) geometry
FROM ST_READOSM('{output_path}') nodes
SEMI JOIN matching_ways_with_nodes_refs
ON nodes.id = matching_ways_with_nodes_refs.ref
WHERE kind = 'node';

SELECT * FROM required_nodes_with_geometries;""")

┌────────────┬──────────────────────────────────────────────┐
│     id     │                   geometry                   │
│   int64    │                   geometry                   │
├────────────┼──────────────────────────────────────────────┤
│   25565560 │ POINT (8.8040912 53.0725385)                 │
│   25565562 │ POINT (8.8020732 53.0716892)                 │
│   25565567 │ POINT (8.800344800000001 53.0706247)         │
│   25580780 │ POINT (8.848536600000001 52.995662800000005) │
│   25581505 │ POINT (8.846047200000001 52.982568900000004) │
│   25600001 │ POINT (8.8289722 52.983601500000006)         │
│   25600054 │ POINT (8.8183085 52.971043400000006)         │
│   25603492 │ POINT (9.893581000000001 52.350886100000004) │
│   25607067 │ POINT (9.6689749 53.464789700000004)         │
│   25607083 │ POINT (9.667208 53.463391900000005)          │
│       ·    │                  ·                           │
│       ·    │                  ·                           │
│       

In [42]:
duck.sql("""CREATE or replace TEMP TABLE matching_ways_linestrings AS
SELECT
    matching_ways.id,
    matching_ways.name,
         matching_ways.highway,
            matching_ways.maxspeed,
            matching_ways.oneway,
            matching_ways.oneway_bus,
            matching_ways.area,
            matching_ways.access,
    st_transform(ST_MakeLine(list(nodes.geometry ORDER BY ref_idx ASC)), 'EPSG:4326', 'EPSG:25832') linestring
FROM matching_ways
JOIN matching_ways_with_nodes_refs
ON matching_ways.id = matching_ways_with_nodes_refs.id
JOIN required_nodes_with_geometries nodes
ON matching_ways_with_nodes_refs.ref = nodes.id
GROUP BY 1, 2, 3,4,5,6,7,8;

SELECT * FROM matching_ways_linestrings;""")

┌───────────┬──────────────────┬──────────────┬──────────┬─────────┬────────────┬─────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    id     │       name       │   highway    │ maxspeed │ oneway  │ oneway_bus │  area 

In [40]:
duck.sql("from st_drivers() order by short_name").df()

,short_name,long_name,can_create,can_copy,can_open,help_url
0,AVCBin,Arc/Info Binary Coverage,False,False,True,https://gdal.org/drivers/vector/avcbin.html
1,AVCE00,Arc/Info E00 (ASCII) Coverage,False,False,True,https://gdal.org/drivers/vector/avce00.html
2,AmigoCloud,AmigoCloud,True,False,True,https://gdal.org/drivers/vector/amigocloud.html
3,CAD,AutoCAD Driver,False,False,True,https://gdal.org/drivers/vector/cad.html
4,CSV,Comma Separated Value (.csv),True,False,True,https://gdal.org/drivers/vector/csv.html
5,CSW,OGC CSW (Catalog Service for the Web),False,False,True,https://gdal.org/drivers/vector/csw.html
6,Carto,Carto,True,False,True,https://gdal.org/drivers/vector/carto.html
7,DGN,Microstation DGN,True,False,True,https://gdal.org/drivers/vector/dgn.html
8,DXF,AutoCAD DXF,True,False,True,https://gdal.org/drivers/vector/dxf.html
9,EDIGEO,French EDIGEO exchange format,False,False,True,https://gdal.org/drivers/vector/edigeo.html


In [ ]:
duck.sql("""copy matching_ways_linestrings to 'out/niedersachsen_highways_geoparquet.gdb' (FORMAT gdal, 
            DRIVER 'OpenFileGDB', 
         SRS 'EPSG:25832', GEOMETRY_TYPE 'LINESTRING');""")